# Reconhece padrões nos testos das leis

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
from sklearn.datasets import fetch_20newsgroups

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
import laws_ai.db as db
db = db.base()

In [3]:
texts = db.texts.find()
fd_texts = pd.DataFrame(list(texts))
fd_texts['cat'] = 0

In [4]:
categories = ["iss","inss","pis","is","cofins","icms"]

In [35]:
from sklearn.model_selection import train_test_split
training_df, test_df = train_test_split(fd_texts, test_size=0.2)
print('total texts in train:',len(training_df))
print('total texts in test:',len(test_df))

total texts in train: 56808
total texts in test: 14203


In [36]:
vocab = Counter()

for text in training_df['text']:
    for word in text.split(' '):
        vocab[word.lower()]+=1
        
for text in test_df['text']:
    for word in text.split(' '):
        vocab[word.lower()]+=1

print("Total words:",len(vocab))

Total words: 68338


In [26]:
total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i
        
    return word2index

word2index = get_word_2_index(vocab)

print("Index of the word 'inss':",word2index['inss'])

Index of the word 'inss': 4066


In [40]:
def text_to_vector(text):
    layer = np.zeros(total_words,dtype=float)
    for word in text.split(' '):
        layer[word2index[word.lower()]] += 1
        
    return layer

def category_to_vector(category):
    y = np.zeros((6),dtype=float) 
    for i in range(len(y)):
        v = 0
        if (category == i): v = 1
        y[i] = v
        
    return y

In [42]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.text[i*batch_size:i*batch_size+batch_size]
    categories = df.cat[i*batch_size:i*batch_size+batch_size]
    
    for text in texts:
        layer = text_to_vector(text) 
        batches.append(layer)
        
    for category in categories:
        y = category_to_vector(category)
        results.append(y)  
     
    return np.array(batches),np.array(results)

In [47]:
print("Each batch has 100 texts and each matrix has x elements (words):",get_batch(training_df,1,100)[0].shape)

Each batch has 100 texts and each matrix has x elements (words): (100, 68338)


In [48]:
print("Each batch has 100 labels and each matrix has 6 elements (6 categories):",get_batch(test_df,1,100)[1].shape)

Each batch has 100 labels and each matrix has 6 elements (6 categories): (100, 6)


In [54]:
# Parameters
learning_rate = 0.01
training_epochs = 10
batch_size = 150
display_step = 1

# Network Parameters
n_hidden_1 = 100      # 1st layer number of features
n_hidden_2 = 100       # 2nd layer number of features
n_input = total_words # Words in vocab
n_classes = 6        # Categories: graphics, sci.space and baseball

input_tensor = tf.placeholder(tf.float32,[None, n_input],name="input")
output_tensor = tf.placeholder(tf.float32,[None, n_classes],name="output") 

In [55]:
def multilayer_perceptron(input_tensor, weights, biases):
    layer_1_multiplication = tf.matmul(input_tensor, weights['h1'])
    layer_1_addition = tf.add(layer_1_multiplication, biases['b1'])
    layer_1 = tf.nn.relu(layer_1_addition)
    
    # Hidden layer with RELU activation
    layer_2_multiplication = tf.matmul(layer_1, weights['h2'])
    layer_2_addition = tf.add(layer_2_multiplication, biases['b2'])
    layer_2 = tf.nn.relu(layer_2_addition)
    
    # Output layer 
    out_layer_multiplication = tf.matmul(layer_2, weights['out'])
    out_layer_addition = out_layer_multiplication + biases['out']
    
    return out_layer_addition

In [56]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
prediction = multilayer_perceptron(input_tensor, weights, biases)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=output_tensor))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Initializing the variables
init = tf.global_variables_initializer()

# [NEW] Add ops to save and restore all the variables
saver = tf.train.Saver()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(training_df.text)/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x,batch_y = get_batch(training_df,i,batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            c,_ = sess.run([loss,optimizer], feed_dict={input_tensor: batch_x,output_tensor:batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "loss=", \
                "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    total_test_data = len(test_df.cat)
    batch_x_test,batch_y_test = get_batch(test_df,0,total_test_data)
    print("Accuracy:", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))
    
    # [NEW] Save the variables to disk
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

Epoch: 0001 loss= 7.391854493
Epoch: 0002 loss= 0.000000000
Epoch: 0003 loss= 0.000000000
Epoch: 0004 loss= 0.000000000
Epoch: 0005 loss= 0.000000000
Epoch: 0006 loss= 0.000000000
Epoch: 0007 loss= 0.000000000
Epoch: 0008 loss= 0.000000000
Epoch: 0009 loss= 0.000000000


## Making a prediction

In [ ]:
# Get a text to make a prediction
text_for_prediction = newsgroups_test.data[5]

In [ ]:
print('text',text_for_prediction)